In [1]:
import imperial_materials_simulation as ims

### Minimal Use Example

In [2]:
sim = ims.Simulation(n_atoms=22, starting_temperature=800, microstructure=ims.starting_microstructure)
sim.display(live_display_interval=10_000) #updates live as simulations are run

In [3]:
sim.NVT_run(n_steps=100_000, temperature=1200, gamma=0.005, integrator='OVRVO')
sim.relax_run(n_steps=50_000, step_size=0.03)
sim.MMC_run(n_steps=500_000, temperature=800, random_scale=0.1)
sim.NVE_run(n_steps=100_000, temperature=600, gamma=0.005, integrator='Verlet')
sim.save("simulation 100k")

### Data Collection Example

**Must be copied into a .py file to work**

In [ ]:
import imperial_materials_simulation as ims
import concurrent.futures as cf #Python standard multiprocessing library 

def sample_workflow(temperature: int) -> list[ims.Simulation, int]:
    #reduce size of simulation files by increasing the microstructure logging interval
    simulation = ims.Simulation(n_atoms=22, starting_temperature=temperature, microstructure_logging_interval=1_000)
    simulation.NVT_run(n_steps=100_000, temperature=temperature)
    simulation.MMC_run(n_steps=500_000, temperature=temperature)
    return simulation, temperature

if __name__ == '__main__':

    #only works in a .py file
    with cf.ProcessPoolExecutor() as executor:
        jobs = [executor.submit(sample_workflow, temperature) for temperature in range(100, 900, 100)]
        
        for job in cf.as_completed(jobs):
            simulation, temperature = job.result()
            print(f'simulation at {temperature}k finished')
            simulation.save(f'simulation {temperature}k') 

### Detailed Analysis Example

In [2]:
sim = ims.load_simulation("simulation 150k")
#all data is stored as Pandas DataFrames

In [3]:
run = 1
step = 95_000

In [4]:
sim.run_data

,run,type,n_steps,T,KE,PE_bonding,PE_non_bonding,PE_total,F_rms,L_end_to_end
0,0,init,0,150.0,0.571678,2.683128e-28,-0.126862,-0.126862,0.000948,32.130000
0,1,NVT,100000,150.0,0.415898,1.175078e-01,-0.306645,-0.189138,0.357985,5.102897
0,2,MMC,300000,150.0,0.415898,2.147332e-01,-0.361663,-0.146930,0.462137,6.392735


In [5]:
sim.step_data[run].to_csv('NVT Step Data') #exporting to Excel
sim.step_data[run].head()

,step,T_actual,PE_total,KE_total,F_rms,L_end_to_end
0,0,212.781595,-0.123080,0.577587,0.076052,32.132831
1,1,200.519294,-0.111808,0.544302,0.152367,32.135541
2,2,192.923963,-0.095277,0.523684,0.219676,32.138465
3,3,193.324065,-0.075038,0.524770,0.280331,32.141691
4,4,180.007398,-0.052675,0.488623,0.334187,32.145000


In [6]:
sim.microstructures[run][step]

,x,y,z
0,-2.740764,-4.553299,2.952537
1,-2.176218,-3.279805,3.573229
2,-0.969065,-2.510495,4.129735
3,0.485907,-2.379651,3.741355
4,1.382861,-1.734796,2.670601
5,1.409916,-1.665365,1.097391
6,2.426691,-1.375399,-0.099301
7,3.294976,-0.484090,-0.992181
8,3.777702,0.486540,-2.033928
9,3.237654,1.827686,-2.571190
